In [4]:
from sqlalchemy.engine import create_engine
from sqlalchemy.orm.session import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.schema import Column, ForeignKey
from sqlalchemy.types import Integer, Text
from sqlalchemy.orm import relationship

In [33]:
# 환경 초기화 작업
sess.close() # 세션 닫기
base.registry.dispose() # 베이스 비우기
base.metadata.clear() # 메타데이터 비우기
engine.dispose() # 엔진 지우기

2024-03-19 09:44:59,071 INFO sqlalchemy.engine.Engine ROLLBACK


In [34]:
engine = create_engine('sqlite:///:memory:', echo=True)
session = sessionmaker(engine)
sess = session()

In [35]:
base = declarative_base()

In [36]:
# base.metadata.reflect(engine)

In [37]:
class Artist(base):
    __tablename__ = 'T_ARTIST'
    __table_args__ = {'extend_existing':True}  # 수정 클래스 덮어씌우는

    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    albums = relationship("Album", back_populates="artist", uselist=True)


class Album(base):
    __tablename__ = 'T_ALBUM'
    __table_args__ = {'extend_existing':True}

    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    fk = Column('FK', None, ForeignKey(Artist.pk))
    artist = relationship("Artist", back_populates="albums", uselist=False)


In [38]:
base.metadata.create_all(engine)

2024-03-19 09:45:05,289 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-19 09:45:05,291 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_ARTIST")
2024-03-19 09:45:05,292 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-19 09:45:05,294 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("T_ARTIST")
2024-03-19 09:45:05,294 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-19 09:45:05,295 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_ALBUM")
2024-03-19 09:45:05,295 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-19 09:45:05,296 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("T_ALBUM")
2024-03-19 09:45:05,297 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-19 09:45:05,298 INFO sqlalchemy.engine.Engine 
CREATE TABLE "T_ARTIST" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT, 
	PRIMARY KEY ("PK")
)


2024-03-19 09:45:05,299 INFO sqlalchemy.engine.Engine [no key 0.00059s] ()
2024-03-19 09:45:05,300 INFO sqlalchemy.engine.Engine 
CREATE TABLE "T_ALBUM" (
	"PK" I

In [43]:
base.__subclasses__(), base.metadata.tables, sess.dirty

([__main__.Artist, __main__.Album],
 FacadeDict({'T_ARTIST': Table('T_ARTIST', MetaData(), Column('PK', Integer(), table=<T_ARTIST>, primary_key=True, nullable=False), Column('NAME', Text(), table=<T_ARTIST>), schema=None), 'T_ALBUM': Table('T_ALBUM', MetaData(), Column('PK', Integer(), table=<T_ALBUM>, primary_key=True, nullable=False), Column('NAME', Text(), table=<T_ALBUM>), Column('FK', Integer(), ForeignKey('T_ARTIST.PK'), table=<T_ALBUM>), schema=None)}),
 IdentitySet([]))

In [44]:
a = Artist(name='수지')

sess.add(a)
sess.commit()

sess.add(Album(name="수지앨범", fk=a.pk))
sess.commit()

2024-03-19 09:47:54,118 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-19 09:47:54,122 INFO sqlalchemy.engine.Engine INSERT INTO "T_ARTIST" ("NAME") VALUES (?)
2024-03-19 09:47:54,123 INFO sqlalchemy.engine.Engine [generated in 0.00082s] ('수지',)
2024-03-19 09:47:54,124 INFO sqlalchemy.engine.Engine COMMIT
2024-03-19 09:47:54,125 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-19 09:47:54,127 INFO sqlalchemy.engine.Engine SELECT "T_ARTIST"."PK" AS "T_ARTIST_PK", "T_ARTIST"."NAME" AS "T_ARTIST_NAME" 
FROM "T_ARTIST" 
WHERE "T_ARTIST"."PK" = ?
2024-03-19 09:47:54,128 INFO sqlalchemy.engine.Engine [generated in 0.00150s] (1,)
2024-03-19 09:47:54,132 INFO sqlalchemy.engine.Engine INSERT INTO "T_ALBUM" ("NAME", "FK") VALUES (?, ?)
2024-03-19 09:47:54,134 INFO sqlalchemy.engine.Engine [generated in 0.00261s] ('수지앨범', 1)
2024-03-19 09:47:54,135 INFO sqlalchemy.engine.Engine COMMIT


In [45]:
sess.query(Artist).join(Album).all()

2024-03-19 09:47:55,499 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-19 09:47:55,507 INFO sqlalchemy.engine.Engine SELECT "T_ARTIST"."PK" AS "T_ARTIST_PK", "T_ARTIST"."NAME" AS "T_ARTIST_NAME" 
FROM "T_ARTIST" JOIN "T_ALBUM" ON "T_ARTIST"."PK" = "T_ALBUM"."FK"
2024-03-19 09:47:55,508 INFO sqlalchemy.engine.Engine [generated in 0.00085s] ()


In [46]:
base.__subclasses__()

[__main__.Artist, __main__.Album]

In [50]:
del base
base = declarative_base()

In [51]:
base.metadata.tables

FacadeDict({})

In [52]:
class Artist(base):
    __table__ = base.metadata.tables['T_ARTIST']

    pk = base.metadata.tables['T_ARTIST'].c.PK
    name = base.metadata.tables['T_ARTIST'].c.NAME
    albums = relationship("Album", back_populates="artist", uselist=True)

class Album(base):
    __table__ = base.metadata.tables['T_ALBUM']

    pk = base.metadata.tables['T_ALBUM'].c.PK
    name = base.metadata.tables['T_ALBUM'].c.NAME
    fk = base.metadata.tables['T_ALBUM'].c.FK
    artist = relationship("Artist", back_populates="albums", uselist=False)


KeyError: 'T_ARTIST'

In [14]:
a = sess.query(Artist).all()[0]

PendingRollbackError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (sqlite3.OperationalError) no such table: T_ARTIST
[SQL: INSERT INTO "T_ARTIST" ("NAME") VALUES (?)]
[parameters: ('수지',)]
(Background on this error at: https://sqlalche.me/e/14/e3q8) (Background on this error at: https://sqlalche.me/e/14/7s2a)

In [ ]:
a.albums[0].name

In [ ]:
a.albums.appned(Album(name='수지앨범2', fk=a.pk))

In [ ]:
session.dirty, sess.is_modified()

In [18]:
class Artist(base):
    __table__ = base.metadata.tables['T_ARTIST']

    pk = base.metadata.tables['T_ARTIST'].c.PK
    name = base.metadata.tables['T_ARTIST'].c.NAME
    albums = relationship("Album", back_populates="artist", uselist=True)

    def addAlbum(self, s, name):
        s.add(Album(name, fk=self.pk))
        s.commit()

    def delAlbum(self, s, name):
        album = list(filter(lambda a:a.name == name, albums))[0]
        self.albums.remove(album)
        s.commit()

class Album(base):
    __table__ = base.metadata.tables['T_Album']

    pk = base.metadata.tables['T_Album'].c.PK
    name = base.metadata.tables['T_Album'].c.NAME
    fk = base.metadata.tables['T_Album'].c.FK
    artist = relationship("Artist", back_populates="albums", uselist=False)

KeyError: 'T_ARTIST'

In [ ]:
a = sess.query(Aritst).all()[0]

In [ ]:
a.addAlbum(sess, "수지앨범3")

In [ ]:
len(a.albums), a.albums[-1].name

In [ ]:
# 포스팅             + 해시태그 (반드시 풀에 존재)
                    +1   =1
Post                Hashtag
Post.addTags(....)  Tag.plus(), Tag.minus

In [ ]:
# M(model)VC 패턴
# M(model)VVM 패턴